In [1]:
# Hidden cell for imports
from dataclasses import dataclass
import pychor
import galois
import numpy as np
from typing import List

@pychor.local_function
def shamir_share(v, t, n):
    # Step 1: pick t-1 random coefficients
    coefficients = [GF.Random() for _ in range(t-1)]

    # Step 2: construct the polynomial
    coefficients.append(GF(v))
    poly = galois.Poly(GF(coefficients))

    # Step 3: compute the shares
    shares = [(GF(x), poly(GF(x))) for x in range(1, n+1)]
    return shares

@pychor.local_function
def shamir_reconstruct(shares):
    # Step 1: recover the polynomial
    xs = GF([s[0] for s in shares])
    ys = GF([s[1] for s in shares])
    poly = galois.lagrange_poly(xs, ys)

    # Step 2: evaluate f(0) to recover the secret
    secret = poly(0)
    return secret

@pychor.local_function
def add_shares(a, b):
    x1, y1 = a
    x2, y2 = b
    assert x1 == x2
    return x1, y1 + y2

@pychor.local_function
def mult_shares(a, b):
    x1, y1 = a
    x2, y2 = b
    assert x1 == x2
    return x1, y1 * y2

# The BGW Protocol

The Ben-Or, Goldwasser, and Wigderson protocol {cite}`todo` is another classic MPC protocol, this time employing Shamir secret sharing and arithmetic circuits. The BGW protocol performs multiplication using the limited multiplicative homomorphism of Shamir shares. The BGW protocol naturally supports more than 2 parties, and actually *requires* at least 3 parties for the standard variant.

````{admonition} Further reading: BGW Protocol
:class: seealso

For more information on the BGW protocol, see **Section 3.3 of [Pragmatic MPC](https://securecomputation.org/)**.
````

The basic ideas behind the BGW protocol are:
- All values are field elements in $GF(p)$ for some (usually large) prime $p$
- Parties hold Shamir secret shares of values, and shares are built as described in Chapter 9
- Parties perform addition locally, via the additive homomorphism of the shares
- Parties perform multiplication using the limited multiplicative homomorphism of the shares, with a *degree reduction* step to avoid the threshold $t$ growing too large

We first present the protocol for multiplication, then build a BGW-based `SecInt` class that implements the BGW protocol.

## BGW: Input

TODO

In [13]:
GF = galois.GF(2**31-1)
parties = [pychor.Party(f'p{i}') for i in range(1, 6)]
t = 3
n = len(parties)

In [18]:
def protocol_bgw_input(v, owner):
    # Step 1: Create Shamir shares
    shares = shamir_share(v, t, n).unlist(n)

    # Step 2: send each share to its owner
    shares_dict = {}
    for share, p in zip(shares, parties):
        share.send(owner, p)
        shares_dict[p] = share
    return shares_dict

In [9]:
with pychor.LocalBackend():
    p1 = parties[0]
    shares = protocol_bgw_input(p1.constant(5), p1)

    for p, v in shares.items():
        print(p, v)

p1 (GF(1, order=2147483647), GF(1975602375, order=2147483647))@{p1}
p2 (GF(2, order=2147483647), GF(555770634, order=2147483647))@{p1, p2}
p3 (GF(3, order=2147483647), GF(35472076, order=2147483647))@{p1, p3}
p4 (GF(4, order=2147483647), GF(414706701, order=2147483647))@{p1, p4}
p5 (GF(5, order=2147483647), GF(1693474509, order=2147483647))@{p1, p5}


## BGW: Reveal

TODO

In [14]:
def protocol_bgw_reveal(shares):
    # Step 1: Broadcast shares to all parties
    for p1, share in shares.items():
        for p2 in parties:
            share.send(p1, p2)

    # Step 2: reconstruct the secret
    reconstructed = shamir_reconstruct(list(shares.values()))
    return reconstructed

In [17]:
with pychor.LocalBackend():
    p1 = parties[0]
    v = p1.constant(5)
    shares = protocol_bgw_input(v, p1)
    print('Shares:')
    for p, v in shares.items():
        print(p, v)

    revealed = protocol_bgw_reveal(shares)
    print('Revealed value:', revealed)

Shares:
p1 (GF(1, order=2147483647), GF(1639984933, order=2147483647))@{p1}
p2 (GF(2, order=2147483647), GF(703249126, order=2147483647))@{p1, p2}
p3 (GF(3, order=2147483647), GF(1484759878, order=2147483647))@{p1, p3}
p4 (GF(4, order=2147483647), GF(1837033542, order=2147483647))@{p1, p4}
p5 (GF(5, order=2147483647), GF(1760070118, order=2147483647))@{p1, p5}
Revealed value: 5@{p1, p2, p4, p5, p3}


## BGW: Multiplication



In [61]:
@pychor.local_function
def get_y_coord(share):
    return share[1]

def protocol_bgw_mult(x_shares, y_shares):
    # Step 1: multiply shares locally
    z_shares = {p: mult_shares(x_shares[p], y_shares[p]) for p in parties}

    # Step 2: re-share the high-degree shares
    q_is = {p: protocol_bgw_input(get_y_coord(z_shares[p]), p) for p in parties}

    # Step 3: perform degree reduction
    Vi = np.linalg.inv(GF(np.vander(range(1,len(parties)+1), increasing=True)))
    lambda_is = Vi[0]

    @pychor.local_function
    def degree_reduce(q_is):
        pass

    for p, v in q_is.items():
        print(p)
        for p, v in v.items():
            print('  ', p, v)

    return z_shares

In [62]:
with pychor.LocalBackend():
    p1 = parties[0]
    p2 = parties[1]
    x = protocol_bgw_input(p1.constant(5), p1)
    y = protocol_bgw_input(p2.constant(3), p2)
    z = protocol_bgw_mult(x, y)
    print(protocol_bgw_reveal(z))

p1
   p1 (GF(1, order=2147483647), GF(1592242047, order=2147483647))@{p1}
   p2 (GF(2, order=2147483647), GF(1485727880, order=2147483647))@{p1, p2}
   p3 (GF(3, order=2147483647), GF(118097350, order=2147483647))@{p1, p3}
   p4 (GF(4, order=2147483647), GF(1784317751, order=2147483647))@{p1, p4}
   p5 (GF(5, order=2147483647), GF(41938142, order=2147483647))@{p1, p5}
p2
   p1 (GF(1, order=2147483647), GF(832898176, order=2147483647))@{p1, p2}
   p2 (GF(2, order=2147483647), GF(1841725787, order=2147483647))@{p2}
   p3 (GF(3, order=2147483647), GF(1382850720, order=2147483647))@{p2, p3}
   p4 (GF(4, order=2147483647), GF(1603756622, order=2147483647))@{p4, p2}
   p5 (GF(5, order=2147483647), GF(356959846, order=2147483647))@{p5, p2}
p3
   p1 (GF(1, order=2147483647), GF(982352987, order=2147483647))@{p1, p3}
   p2 (GF(2, order=2147483647), GF(1310782448, order=2147483647))@{p2, p3}
   p3 (GF(3, order=2147483647), GF(1135717113, order=2147483647))@{p3}
   p4 (GF(4, order=2147483647), GF